# Read All Dataset CSV

In [1]:
import os
import csv
import re
import pandas as pd
import numpy as np

In [2]:
dataset_names=[]
X_trains=[]
y_trains=[]
X_tests=[]

# 收集所有資料夾名稱，按照數字順序排序資料夾名稱
for folder_name in os.listdir("./Competition_data"):
    dataset_names.append(folder_name)
dataset_names = sorted(dataset_names, key=lambda x: int(re.search(r'\d+', x).group()))

for folder_name in dataset_names:
    # print(folder_name)
    X_trains.append(pd.read_csv(f"./Competition_data/{folder_name}/X_train.csv",header=0))
    y_trains.append(pd.read_csv(f"./Competition_data/{folder_name}/y_train.csv",header=0))
    X_tests.append(pd.read_csv(f"./Competition_data/{folder_name}/X_test.csv",header=0))
    
    
    
# print(len(dataset_names))
# print(len(X_trains))  # 49, 代表有 49 個 dataFrame (每個資料集各一個)
# print(len(y_trains))
# print(len(X_tests))
# print(X_trains[0].dtypes)
# print(y_trains[0].dtypes)

In [3]:
# print(X_trains[42])

## Data Preprocessing & Feature Engineering

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelBinarizer

# 對每組資料進行處理
for i in range(len(dataset_names)):
    # 將連續型資料和數值型資料標準化
    numerical_df = X_trains[i].select_dtypes(include=['float'])   # 數值型特徵
    categorical_df = X_trains[i].select_dtypes(include=['int'])   # 類別型特徵（可能為類別特徵）
    if len(numerical_df.columns) + len(categorical_df.columns) != len(X_trains[i].columns):
        print('Splitting error')
    # numerical_df --> normalization
    scaler = StandardScaler()
    scaler.fit(numerical_df)
    numerical_s = scaler.transform(numerical_df)
    numerical_df = pd.DataFrame(numerical_s, columns=numerical_df.columns)
    X_trains[i] = pd.concat([numerical_df, categorical_df], axis=1)
    
    
    numerical_df = X_tests[i].select_dtypes(include=['float'])   # 數值型特徵
    categorical_df = X_tests[i].select_dtypes(include=['int'])   # 類別型特徵（可能為類別特徵）
    # 直接照前面用過的 scaler 來分
    numerical_s = scaler.transform(numerical_df)
    numerical_df = pd.DataFrame(numerical_s, columns=numerical_df.columns)
    X_tests[i] = pd.concat([numerical_df, categorical_df], axis=1)

for i in range(len(dataset_names)):
    y_trains[i].iloc[:, 0] = pd.to_numeric(y_trains[i].iloc[:, 0])

In [6]:
# from sklearn.preprocessing import MinMaxScaler, LabelBinarizer

# def preprocess_data(df, scaler=None, label_binarizers=None, columns=None):
#     # 將連續型資料和數值型資料標準化
#     numerical_df = df.select_dtypes(include=['float'])   # 數值型特徵
#     categorical_df = df.select_dtypes(include=['int'])   # 類別型特徵（可能為類別特徵）

#     # numerical_df --> normalization
#     if scaler is None:
#         scaler = MinMaxScaler()
#         scaler.fit(numerical_df)
#     numerical_s = scaler.transform(numerical_df)
#     numerical_df = pd.DataFrame(numerical_s, columns=numerical_df.columns)

#     # categorial_df --> label binarizer encoding
#     if label_binarizers is None:
#         label_binarizers = {}

#     encoded_cols = []
#     for col in categorical_df.columns:
#         unique_values = categorical_df[col].nunique()

#         if col not in label_binarizers:
#             c_scaler = LabelBinarizer()
#             c_scaler.fit(categorical_df[col])
#             label_binarizers[col] = c_scaler
#         encoded_df = label_binarizers[col].transform(categorical_df[col])  # 轉成 ndarray

#         # 如果是多類別，轉換成 DataFrame，並加上欄位名稱
#         if encoded_df.shape[1] > 1:
#             encoded_df = pd.DataFrame(encoded_df, columns=[f"{col}_{cls}" for cls in label_binarizers[col].classes_])
#         else:
#             encoded_df = pd.Series(encoded_df.flatten(), name=categorical_df[col].name)
#         encoded_cols.append(encoded_df)

#     encoded_df = pd.concat([categorical_df.drop(columns=categorical_df.columns)] + encoded_cols, axis=1)

#     # 如果是測試資料，補齊缺少的欄位並重新排序
#     if columns is not None:
#         missing_cols = set(columns) - set(encoded_df.columns)
#         for col in missing_cols:
#             encoded_df[col] = 0  # 缺失的欄位補 0
#         encoded_df = encoded_df[columns]  # 重新排序以匹配訓練資料的欄位順序

#     # 合併數值型和類別型資料框
#     processed_df = pd.concat([numerical_df, encoded_df], axis=1)
#     processed_df = processed_df[columns] if columns is not None else processed_df

#     return processed_df, scaler, label_binarizers

# # 對每組資料進行處理
# for i in range(len(dataset_names)):
#     X_trains[i], n_scaler, label_binarizers = preprocess_data(X_trains[i])
#     X_tests[i], _, _ = preprocess_data(X_tests[i], scaler=n_scaler, label_binarizers=label_binarizers, columns=X_trains[i].columns)


In [7]:
# for i in range(len(dataset_names)):
#     missing_cols = set(X_trains[i].columns) - set(X_tests[i].columns)
#     print(missing_cols)

## train test split & build Model
You can select an appropriate model and perform corresponding hyperparameter tuning.

In [8]:
# !pip install tensorflow

In [29]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
# from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.metrics import roc_auc_score
# from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras.regularizers import l2

In [30]:
# models=[]
# avg_auc = 0
# avg_train = 0
# for i in range(len(dataset_names)):
#     # 這邊做一下 stratify
#     tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
#         X_trains[i], y_trains[i], test_size=0.2, random_state=42, stratify=y_trains[i]
#     )
#     # Random Forest (適合這次的小資料集)
#     model = RandomForestClassifier(
#         n_estimators=400, n_jobs=1, random_state=1, max_depth=7,  
#     )

#     # # Neural Net
#     # model = MLPClassifier(
#     #     hidden_layer_sizes=(128,64),
#     #     activation="relu",
#     #     random_state=1,
#     #     alpha=0.01
#     # )
    
#     # # XGBoost (好像比較容易過擬合，適合大資料集) 
#     # model = XGBClassifier(
#     #     n_estimators=200, learning_rate=0.1, gamma=0.8,
#     # )
    
#     model.fit(tmp_X_train, tmp_y_train.squeeze())
    
#     tmp_y_prob = model.predict_proba(tmp_X_test)[:, 1]
#     auc = roc_auc_score(tmp_y_test, tmp_y_prob)
#     # print(f'auc of dataset {i:2}: \t{auc}')
#     avg_auc += auc
#     models.append(model)
    
# print(avg_auc / len(dataset_names))



In [31]:
models=[]
avg_auc = 0
avg_train = 0
for i in range(len(dataset_names)):
    # 這邊做一下 stratify
    tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(
        X_trains[i], y_trains[i], test_size=0.2, random_state=42, stratify=y_trains[i]
    )
    

    
#     # Random Forest (適合這次的小資料集)
#     model = RandomForestClassifier(
#         n_estimators=400, n_jobs=1, random_state=1, max_depth=7,  
#     )

#     #tensorflow(1)(0.820698)
#     model = Sequential()
#     model.add(Dense(units=16, activation='relu', input_shape=(X_trains[i].shape[1],)))
#     model.add(Dense(1, activation='sigmoid'))
#     adam = keras.optimizers.Adam(learning_rate=0.001)
#     model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])
    
#     model.fit(tmp_X_train, tmp_y_train.squeeze(), epochs=100)
    
    #tensorflow(2)(0.842297)
    model = Sequential()
    model.add(Dense(units=32, activation='relu', input_shape=(X_trains[i].shape[1],), kernel_regularizer=l2(0.01)))
    model.add(Dense(1, activation='sigmoid'))
    adam = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])
    
    model.fit(tmp_X_train, tmp_y_train.squeeze(), epochs=100)
    

    # # Neural Net
    # model = MLPClassifier(
    #     hidden_layer_sizes=(128,64),
    #     activation="relu",
    #     random_state=1,
    #     alpha=0.01
    # )
    
    # # XGBoost (好像比較容易過擬合，適合大資料集) 
    # model = XGBClassifier(
    #     n_estimators=200, learning_rate=0.1, gamma=0.8,
    # )
    
#     model.fit(tmp_X_train, tmp_y_train.squeeze())
    
    tmp_y_prob = model.predict(tmp_X_test).ravel()
    auc = roc_auc_score(tmp_y_test, tmp_y_prob)
    # print(f'auc of dataset {i:2}: \t{auc}')
    avg_auc += auc
    models.append(model)
    
print(avg_auc / len(dataset_names))



Epoch 1/100
12/12 [==============================] - 0s 1ms/step - loss: 0.8569 - accuracy: 0.4873
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 0.8135 - accuracy: 0.5296
Epoch 3/100
12/12 [==============================] - 0s 1ms/step - loss: 0.7798 - accuracy: 0.5746
Epoch 4/100
12/12 [==============================] - 0s 1ms/step - loss: 0.7523 - accuracy: 0.6310
Epoch 5/100
12/12 [==============================] - 0s 1ms/step - loss: 0.7316 - accuracy: 0.6535
Epoch 6/100
12/12 [==============================] - 0s 1ms/step - loss: 0.7137 - accuracy: 0.6789
Epoch 7/100
12/12 [==============================] - 0s 1ms/step - loss: 0.6985 - accuracy: 0.7014
Epoch 8/100
12/12 [==============================] - 0s 1ms/step - loss: 0.6840 - accuracy: 0.7239
Epoch 9/100
12/12 [==============================] - 0s 1ms/step - loss: 0.6716 - accuracy: 0.7296
Epoch 10/100
12/12 [==============================] - 0s 1ms/step - loss: 0.6608 - accuracy: 0.7296
Epoch 11/

12/12 [==============================] - 0s 977us/step - loss: 0.4925 - accuracy: 0.7690
Epoch 83/100
12/12 [==============================] - 0s 1ms/step - loss: 0.4919 - accuracy: 0.7662
Epoch 84/100
12/12 [==============================] - 0s 1ms/step - loss: 0.4912 - accuracy: 0.7634
Epoch 85/100
12/12 [==============================] - 0s 1ms/step - loss: 0.4906 - accuracy: 0.7718
Epoch 86/100
12/12 [==============================] - 0s 1ms/step - loss: 0.4898 - accuracy: 0.7690
Epoch 87/100
12/12 [==============================] - 0s 1ms/step - loss: 0.4903 - accuracy: 0.7690
Epoch 88/100
12/12 [==============================] - 0s 1ms/step - loss: 0.4894 - accuracy: 0.7634
Epoch 89/100
12/12 [==============================] - 0s 821us/step - loss: 0.4891 - accuracy: 0.7662
Epoch 90/100
12/12 [==============================] - 0s 1ms/step - loss: 0.4886 - accuracy: 0.7662
Epoch 91/100
12/12 [==============================] - 0s 915us/step - loss: 0.4873 - accuracy: 0.7690
Epoch 9

12/12 [==============================] - 0s 1ms/step - loss: 0.1156 - accuracy: 0.9812
Epoch 64/100
12/12 [==============================] - 0s 1ms/step - loss: 0.1145 - accuracy: 0.9812
Epoch 65/100
12/12 [==============================] - 0s 944us/step - loss: 0.1130 - accuracy: 0.9785
Epoch 66/100
12/12 [==============================] - 0s 948us/step - loss: 0.1115 - accuracy: 0.9812
Epoch 67/100
12/12 [==============================] - 0s 894us/step - loss: 0.1107 - accuracy: 0.9812
Epoch 68/100
12/12 [==============================] - 0s 839us/step - loss: 0.1096 - accuracy: 0.9812
Epoch 69/100
12/12 [==============================] - 0s 1ms/step - loss: 0.1089 - accuracy: 0.9812
Epoch 70/100
12/12 [==============================] - 0s 1ms/step - loss: 0.1075 - accuracy: 0.9785
Epoch 71/100
12/12 [==============================] - 0s 854us/step - loss: 0.1068 - accuracy: 0.9785
Epoch 72/100
12/12 [==============================] - 0s 1ms/step - loss: 0.1058 - accuracy: 0.9812
Epo

2/2 [==============================] - 0s 4ms/step - loss: 0.6399 - accuracy: 0.7636
Epoch 46/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6370 - accuracy: 0.7636
Epoch 47/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6342 - accuracy: 0.7636
Epoch 48/100
2/2 [==============================] - 0s 5ms/step - loss: 0.6312 - accuracy: 0.7636
Epoch 49/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6283 - accuracy: 0.7636
Epoch 50/100
2/2 [==============================] - 0s 5ms/step - loss: 0.6255 - accuracy: 0.7636
Epoch 51/100
2/2 [==============================] - 0s 2ms/step - loss: 0.6224 - accuracy: 0.7636
Epoch 52/100
2/2 [==============================] - 0s 4ms/step - loss: 0.6196 - accuracy: 0.7636
Epoch 53/100
2/2 [==============================] - 0s 2ms/step - loss: 0.6169 - accuracy: 0.7818
Epoch 54/100
2/2 [==============================] - 0s 3ms/step - loss: 0.6139 - accuracy: 0.7818
Epoch 55/100
2/2 [===============

13/13 [==============================] - 0s 1ms/step - loss: 0.3147 - accuracy: 0.9489
Epoch 28/100
13/13 [==============================] - 0s 1ms/step - loss: 0.3090 - accuracy: 0.9489
Epoch 29/100
13/13 [==============================] - 0s 1ms/step - loss: 0.3038 - accuracy: 0.9489
Epoch 30/100
13/13 [==============================] - 0s 1ms/step - loss: 0.2984 - accuracy: 0.9489
Epoch 31/100
13/13 [==============================] - 0s 1ms/step - loss: 0.2938 - accuracy: 0.9489
Epoch 32/100
13/13 [==============================] - 0s 971us/step - loss: 0.2888 - accuracy: 0.9489
Epoch 33/100
13/13 [==============================] - 0s 922us/step - loss: 0.2843 - accuracy: 0.9489
Epoch 34/100
13/13 [==============================] - 0s 1ms/step - loss: 0.2798 - accuracy: 0.9489
Epoch 35/100
13/13 [==============================] - 0s 899us/step - loss: 0.2757 - accuracy: 0.9489
Epoch 36/100
13/13 [==============================] - 0s 1ms/step - loss: 0.2717 - accuracy: 0.9489
Epoch 3

...

5/5 [==============================] - 0s 1ms/step - loss: 0.5302 - accuracy: 0.8714
Epoch 25/100
5/5 [==============================] - 0s 1ms/step - loss: 0.5248 - accuracy: 0.8714
Epoch 26/100
5/5 [==============================] - 0s 1ms/step - loss: 0.5195 - accuracy: 0.8714
Epoch 27/100
5/5 [==============================] - 0s 1ms/step - loss: 0.5145 - accuracy: 0.8714
Epoch 28/100
5/5 [==============================] - 0s 2ms/step - loss: 0.5097 - accuracy: 0.8714
Epoch 29/100
5/5 [==============================] - 0s 1ms/step - loss: 0.5050 - accuracy: 0.8714
Epoch 30/100
5/5 [==============================] - 0s 2ms/step - loss: 0.5007 - accuracy: 0.8714
Epoch 31/100
5/5 [==============================] - 0s 1ms/step - loss: 0.4964 - accuracy: 0.8714
Epoch 32/100
5/5 [==============================] - 0s 2ms/step - loss: 0.4921 - accuracy: 0.8714
Epoch 33/100
5/5 [==============================] - 0s 2ms/step - loss: 0.4884 - accuracy: 0.8714
Epoch 34/100
5/5 [===============

3/3 [==============================] - 0s 2ms/step - loss: 0.8859 - accuracy: 0.7059
Epoch 8/100
3/3 [==============================] - 0s 2ms/step - loss: 0.8649 - accuracy: 0.7059
Epoch 9/100
3/3 [==============================] - 0s 2ms/step - loss: 0.8455 - accuracy: 0.7206
Epoch 10/100
3/3 [==============================] - 0s 1ms/step - loss: 0.8272 - accuracy: 0.7647
Epoch 11/100
3/3 [==============================] - 0s 2ms/step - loss: 0.8094 - accuracy: 0.7647
Epoch 12/100
3/3 [==============================] - 0s 1ms/step - loss: 0.7914 - accuracy: 0.7941
Epoch 13/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7754 - accuracy: 0.8088
Epoch 14/100
3/3 [==============================] - 0s 1ms/step - loss: 0.7606 - accuracy: 0.8088
Epoch 15/100
3/3 [==============================] - 0s 3ms/step - loss: 0.7473 - accuracy: 0.8088
Epoch 16/100
3/3 [==============================] - 0s 2ms/step - loss: 0.7333 - accuracy: 0.8235
Epoch 17/100
3/3 [=================

3/3 [==============================] - 0s 2ms/step - loss: 0.3175 - accuracy: 0.9853
Epoch 91/100
3/3 [==============================] - 0s 2ms/step - loss: 0.3149 - accuracy: 1.0000
Epoch 92/100
3/3 [==============================] - 0s 2ms/step - loss: 0.3123 - accuracy: 1.0000
Epoch 93/100
3/3 [==============================] - 0s 2ms/step - loss: 0.3096 - accuracy: 1.0000
Epoch 94/100
3/3 [==============================] - 0s 2ms/step - loss: 0.3071 - accuracy: 1.0000
Epoch 95/100
3/3 [==============================] - 0s 2ms/step - loss: 0.3043 - accuracy: 1.0000
Epoch 96/100
3/3 [==============================] - 0s 3ms/step - loss: 0.3020 - accuracy: 1.0000
Epoch 97/100
3/3 [==============================] - 0s 2ms/step - loss: 0.2994 - accuracy: 1.0000
Epoch 98/100
3/3 [==============================] - 0s 3ms/step - loss: 0.2970 - accuracy: 1.0000
Epoch 99/100
3/3 [==============================] - 0s 2ms/step - loss: 0.2947 - accuracy: 1.0000
Epoch 100/100
1/1 [==============

6/6 [==============================] - 0s 3ms/step - loss: 0.2106 - accuracy: 0.9630
Epoch 74/100
6/6 [==============================] - 0s 2ms/step - loss: 0.2090 - accuracy: 0.9630
Epoch 75/100
6/6 [==============================] - 0s 2ms/step - loss: 0.2076 - accuracy: 0.9630
Epoch 76/100
6/6 [==============================] - 0s 2ms/step - loss: 0.2061 - accuracy: 0.9630
Epoch 77/100
6/6 [==============================] - 0s 1ms/step - loss: 0.2047 - accuracy: 0.9630
Epoch 78/100
6/6 [==============================] - 0s 1ms/step - loss: 0.2033 - accuracy: 0.9630
Epoch 79/100
6/6 [==============================] - 0s 1ms/step - loss: 0.2019 - accuracy: 0.9630
Epoch 80/100
6/6 [==============================] - 0s 1ms/step - loss: 0.2005 - accuracy: 0.9630
Epoch 81/100
6/6 [==============================] - 0s 2ms/step - loss: 0.1992 - accuracy: 0.9630
Epoch 82/100
6/6 [==============================] - 0s 1ms/step - loss: 0.1979 - accuracy: 0.9630
Epoch 83/100
6/6 [===============

15/15 [==============================] - 0s 964us/step - loss: 0.2049 - accuracy: 0.9370
Epoch 55/100
15/15 [==============================] - 0s 1ms/step - loss: 0.2024 - accuracy: 0.9326
Epoch 56/100
15/15 [==============================] - 0s 1ms/step - loss: 0.2012 - accuracy: 0.9370
Epoch 57/100
15/15 [==============================] - 0s 1ms/step - loss: 0.1986 - accuracy: 0.9370
Epoch 58/100
15/15 [==============================] - 0s 982us/step - loss: 0.1969 - accuracy: 0.9391
Epoch 59/100
15/15 [==============================] - 0s 1ms/step - loss: 0.1955 - accuracy: 0.9348
Epoch 60/100
15/15 [==============================] - 0s 1ms/step - loss: 0.1948 - accuracy: 0.9348
Epoch 61/100
15/15 [==============================] - 0s 1ms/step - loss: 0.1930 - accuracy: 0.9370
Epoch 62/100
15/15 [==============================] - 0s 1ms/step - loss: 0.1919 - accuracy: 0.9391
Epoch 63/100
15/15 [==============================] - 0s 1ms/step - loss: 0.1896 - accuracy: 0.9370
Epoch 64/

## Inference Model

In [32]:
y_predicts=[]
for i in range(len(dataset_names)):
    # print(X_tests[i])
#     print(X_tests[i].shape)
#     print(models[i].layers[0].input_shape)


    y_predict_proba=models[i].predict(X_tests[i]).ravel()
    df = pd.DataFrame(y_predict_proba, columns=['y_predict_proba'])
    y_predicts.append(df)
    

12/12 [==============================] - 0s 1ms/step


## Save result

In [33]:
for idx,dataset_name in enumerate(dataset_names):
    df=y_predicts[idx]
    df.to_csv(f'./Competition_data/{dataset_name}/y_predict.csv', index=False,header=True)

In [89]:
# from sklearn.metrics import roc_auc_score

# test_aucs = []
# for i in range(len(dataset_names)):
#     # 使用模型進行預測，獲得類別 1 的預測機率
#     y_predict_proba = models[i].predict_proba(X_tests[i])[:, 1]
    
#     # 計算 AUC 分數
#     auc = roc_auc_score(y_tests[i], y_predict_proba)
#     print(f'AUC of dataset {i:2}: \t{auc}')
    
#     test_aucs.append(auc)

# # 平均 AUC
# avg_test_auc = sum(test_aucs) / len(test_aucs)
# print("\nAverage Test AUC:", avg_test_auc)
